In [19]:
import pandas as pd

In [20]:
file_path = "eu_bdt-ud-dev.conllu"  # Replace with the actual path to your file

lines = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#'):  # Filter out empty lines and comments
            lines.append(line.split('\t'))

In [21]:
column_names = ['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc']
df = pd.DataFrame(lines, columns=column_names)

In [22]:
df = df.replace({'_': pd.NA})  # Replace '_' with Na

In [23]:
df['feats'] = df['feats'].fillna('')

In [24]:
threshold = 0.8 
df = df.dropna(thresh=len(df) * (1 - threshold), axis=1) 
print("Columns remaining:", df.columns)

Columns remaining: Index(['id', 'form', 'lemma', 'upos', 'feats', 'head', 'deprel'], dtype='object')


In [25]:
tresh=len(df) * (1 - threshold)

In [26]:
axis=1

In [27]:
df.columns

Index(['id', 'form', 'lemma', 'upos', 'feats', 'head', 'deprel'], dtype='object')

In [28]:
lemmas_of_interest = ['izan', 'ukan'] 
df_filtered = df[df['lemma'].isin(lemmas_of_interest)]

In [29]:
df_filtered

,id,form,lemma,upos,feats,head,deprel
7,8,izan,izan,VERB,Aspect=Perf|VerbForm=Part,0,root
15,6,da,izan,AUX,Mood=Ind|Number[abs]=Sing|Person[abs]=3|VerbFo...,5,aux
29,12,zen,izan,AUX,Mood=Ind|Number[abs]=Sing|Person[abs]=3|VerbFo...,11,aux
34,4,den,izan,AUX,Mood=Ind|Number[abs]=Sing|Person[abs]=3|VerbFo...,3,aux
39,2,izan,izan,VERB,Aspect=Perf|VerbForm=Part,4,acl
...,...,...,...,...,...,...,...
24026,6,duten,ukan,VERB,VerbForm=Fin,7,acl
24075,6,diren,izan,AUX,Mood=Ind|Number[abs]=Plur|Person[abs]=3|VerbFo...,4,aux
24081,12,duela,ukan,VERB,Aspect=Prog|Mood=Ind|Number[abs]=Sing|Number[e...,9,advcl
24090,21,izan,izan,AUX,Aspect=Perf|VerbForm=Part,20,cop


In [30]:
import re
def extract_features(feats):
    if pd.isna(feats):
        return []
    return re.split(r'\|=', feats)
    

In [31]:
df_filtered.loc[:, 'feats.list'] = df_filtered['feats'].apply(extract_features) #Creates a colum

/tmp/ipykernel_15345/4061473165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'feats.list'] = df_filtered['feats'].apply(extract_features) #Creates a colum


In [33]:
all_features = []
for feats in df_filtered['feats.list']:
    all_features.extend(feats)
unique_features = set(all_features)